In [1]:
!pip install --upgrade pip
!pip install python-binance

# Option valuation model based on historical prices

In [1]:
import importlib.util
import os
google_colab_spec = importlib.util.find_spec("google") and importlib.util.find_spec("google.colab")
if google_colab_spec:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    path = '/content/drive/MyDrive/Colab Notebooks/Ethereum Prices'
else:
    path = f'{os.environ["HOME"]}/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices'

from datetime import timezone

import optionspricing
import binanceprices

COUNT_YEARS = 10
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'

prices_df = binanceprices.load_prices(path, BINANCE_SYMBOL, COUNT_YEARS)



loading 2014 ............
loading 2015 ............
loading 2016 ............
loading 2017 ............
loading 2018 ............
loading 2019 ............
loading 2020 ............
loading 2021 ............
loading 2022 ............
loading 2023 ............
loading 2024 
interrupting at 2024/01


## Option valuation model: input parameters here

In [4]:
target_period_hours = 1 * 24 + 10
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 7)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size)
option_chain_df

,value_call,value_call_pct,call_ask,value_put,value_put_pct,put_ask
strike,,,,,,
2125.0,122.231721,NaN,0.0590,9.972284,0.004464,0.0013
2150.0,100.743081,NaN,0.0475,13.483644,0.006036,0.0020
2175.0,80.582299,NaN,0.0440,18.322863,0.008202,0.0033
2200.0,62.263859,NaN,0.0210,25.004423,0.011193,0.0055
2225.0,46.561978,NaN,0.0135,34.302542,0.015355,0.0090
2250.0,34.530852,0.015457,0.0075,47.271415,NaN,0.0150
2275.0,25.661484,0.011487,0.0040,63.402048,NaN,NaN
2300.0,18.968047,0.008491,0.0022,81.708610,NaN,NaN
2325.0,13.953752,0.006246,0.0015,101.694316,NaN,0.0545


In [5]:
#
index_put = strikes_universe_size - 2
index_call = strikes_universe_size + 2
#
cost, value = (option_chain_df.iloc[index_put]['put_ask'] + option_chain_df.iloc[index_call]['call_ask'], 
 option_chain_df.iloc[index_put]['value_put_pct'] + option_chain_df.iloc[index_call]['value_call_pct']
)
print(f'target expiry: {target_expiry.astimezone(timezone.utc)} ({remaining_hours} hours left)')
print(f'current price: {underlying_price}')
print(f'trading put {option_chain_df.iloc[index_put].name:.0f} and call {option_chain_df.iloc[index_call].name:.0f}')
print(f'cost: {cost:.3f} / value: {value:.3f}, benefit/cost = {value / cost:.1f}x')
print(f'($) cost: {cost * underlying_price:.2f} / value: {value * underlying_price:.2f}, average gain = {(value - cost) * underlying_price:.2f}')


target expiry: 2024-01-28 08:00:00+00:00 (45 hours left)
current price: 2233.94
trading put 2175 and call 2275
cost: 0.007 / value: 0.020, benefit/cost = 2.7x
($) cost: 16.31 / value: 43.98, average gain = 27.68


In [9]:
strategy_value_pct = trading_model.valuation[f"put_value_pct_{index_put}"] + trading_model.valuation[f"call_value_pct_{index_call}"]

In [10]:
strategy_value_pct

dateTime
2021-07-01 00:00:00    0.036346
2021-07-01 01:00:00    0.001498
2021-07-01 02:00:00    0.009515
2021-07-01 03:00:00    0.000000
2021-07-01 04:00:00    0.000000
                         ...   
2023-12-29 22:00:00    0.000000
2023-12-29 23:00:00    0.000000
2023-12-30 00:00:00    0.000000
2023-12-30 01:00:00    0.000000
2023-12-30 02:00:00    0.000000
Length: 21884, dtype: float64